# Notebook 01 -- Exploratory Data Analysis

## Verizon Cell Tower Lease Fair Market Value Estimation

This notebook provides a **comprehensive EDA** of the synthetic cell tower lease dataset
(5,000 sites) that underpins the FMV estimation pipeline. We follow the analysis plan
laid out in Sections 5.2.1 through 5.2.6 of the solution architecture.

### Sections

| Section | Description |
|---------|-------------|
| 1. Setup and Data Load | Imports, load parquet, basic shape/dtypes |
| 2. Univariate Analysis (5.2.1) | Distributions of rent, log-rent, rent by property type |
| 3. Bivariate Analysis (5.2.2) | Scatter plots of rent vs key predictors |
| 4. Spatial Analysis (5.2.3) | US map scatter plots, rent by state/metro |
| 5. Cross-Property-Type (5.2.4) | Kruskal-Wallis, ANOVA, variance decomposition |
| 6. Statistical Tests (5.2.5) | Shapiro-Wilk, correlations, VIF, outliers |
| 7. Temporal Analysis (5.2.6) | Escalation drift analysis |
| 8. Summary | Key findings |

---
## 1. Setup and Data Load

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import shapiro, kruskal, f_oneway, pearsonr, spearmanr
from itertools import combinations
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams.update({
    'figure.dpi': 120,
    'figure.facecolor': 'white',
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
})

PROP_TYPES = ['commercial', 'rooftop', 'vacant_land', 'residential',
              'government', 'agricultural', 'industrial']
PROP_COLORS = dict(zip(PROP_TYPES, sns.color_palette('Set2', len(PROP_TYPES))))

print('All imports loaded.')

In [ ]:
DATA_PATH = '../data/synthetic/synthetic_5k_sites.parquet'

df = pd.read_parquet(DATA_PATH)
print(f'Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')
print(f'Memory usage: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB')
print()
df.dtypes

In [ ]:
df.describe(include='all').T

In [ ]:
missing = df.isnull().sum()
if missing.sum() == 0:
    print('No missing values in any column.')
else:
    print('Columns with missing values:')
    print(missing[missing > 0])

---
## 2. Section 5.2.1 -- Univariate Analysis

We examine the marginal distributions of the target variable (`current_monthly_rent`),
its log-transform, rent stratified by property type, lease age, and key market features.
The goal is to assess skewness, identify potential outliers, and determine whether a
log-transform is appropriate for modelling.

### 2.1 Distribution of current_monthly_rent

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Histogram
axes[0].hist(df['current_monthly_rent'], bins=80, color='steelblue',
             edgecolor='white', alpha=0.85)
axes[0].axvline(df['current_monthly_rent'].median(), color='red', ls='--',
                lw=1.5, label=f'Median = ${df["current_monthly_rent"].median():,.0f}')
axes[0].set_title('Current Monthly Rent -- Histogram')
axes[0].set_xlabel('$/month')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Boxplot
bp = axes[1].boxplot(df['current_monthly_rent'], vert=True, patch_artist=True,
                     boxprops=dict(facecolor='steelblue', alpha=0.7),
                     medianprops=dict(color='red', lw=2))
axes[1].set_title('Current Monthly Rent -- Boxplot')
axes[1].set_ylabel('$/month')
axes[1].set_xticklabels(['All Sites'])

plt.suptitle('Section 5.2.1: Distribution of Current Monthly Rent', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print(f'Mean:   ${df["current_monthly_rent"].mean():,.0f}')
print(f'Median: ${df["current_monthly_rent"].median():,.0f}')
print(f'Std:    ${df["current_monthly_rent"].std():,.0f}')
print(f'Skew:   {df["current_monthly_rent"].skew():.2f}')
print(f'Kurt:   {df["current_monthly_rent"].kurtosis():.2f}')

### 2.2 Distribution of log(current_monthly_rent) -- Normality Check

Cell tower rents are expected to be right-skewed. A log-transform should approximately
normalise the distribution, which is important because:
1. Our FMV generating function is log-linear
2. Many regression models assume approximate normality of residuals
3. Percentage-based error metrics are more meaningful in log-space

In [ ]:
log_rent = np.log(df['current_monthly_rent'])

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Histogram with normal overlay
axes[0].hist(log_rent, bins=80, color='coral', edgecolor='white', alpha=0.8, density=True)
xmin, xmax = axes[0].get_xlim()
x_grid = np.linspace(xmin, xmax, 200)
mu, sigma = log_rent.mean(), log_rent.std()
axes[0].plot(x_grid, stats.norm.pdf(x_grid, mu, sigma), 'k-', lw=2,
             label=f'N({mu:.2f}, {sigma:.2f}$^2$)')
axes[0].set_title('log(Current Monthly Rent) -- Histogram + Normal Fit')
axes[0].set_xlabel('log($/month)')
axes[0].set_ylabel('Density')
axes[0].legend()

# Q-Q plot
stats.probplot(log_rent, dist='norm', plot=axes[1])
axes[1].set_title('Q-Q Plot: log(Rent) vs Normal')
axes[1].get_lines()[0].set_markersize(2)
axes[1].get_lines()[0].set_alpha(0.5)

plt.suptitle('Section 5.2.1: Log-Rent Normality Check', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print(f'log(rent) skewness: {log_rent.skew():.3f}')
print(f'log(rent) kurtosis: {log_rent.kurtosis():.3f}')
print(f'=> Log-transform substantially reduces skewness '
      f'(original: {df["current_monthly_rent"].skew():.2f}).')

### 2.3 Distribution of Rent BY Property Type

We expect rent distributions to differ across property types due to differences in
land value, urban/rural mix, and tower economics. This motivates the cross-property-type
analysis in Section 5.2.4.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Overlaid histograms (log scale for readability)
for pt in PROP_TYPES:
    subset = df.loc[df['property_type'] == pt, 'current_monthly_rent']
    axes[0].hist(np.log(subset), bins=50, alpha=0.45,
                 label=f'{pt} (n={len(subset)})',
                 color=PROP_COLORS[pt], edgecolor='none')
axes[0].set_title('log(Rent) by Property Type -- Overlaid Histograms')
axes[0].set_xlabel('log($/month)')
axes[0].set_ylabel('Frequency')
axes[0].legend(fontsize=7, ncol=2)

# Faceted box plots
order = (df.groupby('property_type')['current_monthly_rent']
           .median().sort_values(ascending=False).index.tolist())
sns.boxplot(data=df, x='property_type', y='current_monthly_rent',
            order=order, palette=PROP_COLORS, ax=axes[1],
            flierprops=dict(marker='o', markersize=2, alpha=0.3))
axes[1].set_title('Current Rent by Property Type -- Box Plots')
axes[1].set_xlabel('Property Type')
axes[1].set_ylabel('$/month')
axes[1].tick_params(axis='x', rotation=30)

plt.suptitle('Section 5.2.1: Rent Distribution by Property Type', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

rent_summary = (df.groupby('property_type')['current_monthly_rent']
                  .agg(['count', 'mean', 'median', 'std', 'min', 'max'])
                  .round(0)
                  .sort_values('median', ascending=False))
rent_summary.columns = ['N', 'Mean ($)', 'Median ($)', 'Std ($)', 'Min ($)', 'Max ($)']
print('\nRent by Property Type:')
rent_summary

### 2.4 Distribution of lease_age_years

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].hist(df['lease_age_years'], bins=50, color='mediumpurple',
             edgecolor='white', alpha=0.85)
axes[0].axvline(df['lease_age_years'].median(), color='red', ls='--', lw=1.5,
                label=f'Median = {df["lease_age_years"].median():.1f} yrs')
axes[0].set_title('Lease Age Distribution')
axes[0].set_xlabel('Years')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Lease age by property type
sns.boxplot(data=df, x='property_type', y='lease_age_years',
            order=PROP_TYPES, palette=PROP_COLORS, ax=axes[1],
            flierprops=dict(marker='o', markersize=2, alpha=0.3))
axes[1].set_title('Lease Age by Property Type')
axes[1].set_xlabel('Property Type')
axes[1].set_ylabel('Years')
axes[1].tick_params(axis='x', rotation=30)

plt.suptitle('Section 5.2.1: Lease Age Distribution', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print(f'Lease age: mean={df["lease_age_years"].mean():.1f}, '
      f'median={df["lease_age_years"].median():.1f}, '
      f'range=[{df["lease_age_years"].min():.1f}, {df["lease_age_years"].max():.1f}]')

### 2.5 De-Escalated Base Rent vs Current Rent

The de-escalated base rent removes the cumulative escalation applied over the lease
term. Comparing these two distributions reveals the magnitude of contractual rent
drift. Large gaps indicate leases that have drifted far from their origination basis.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

ax.hist(np.log(df['current_monthly_rent']), bins=80, alpha=0.6,
        color='steelblue', edgecolor='none', label='log(Current Rent)')
ax.hist(np.log(df['de_escalated_base_rent'].clip(lower=1)), bins=80, alpha=0.6,
        color='coral', edgecolor='none', label='log(De-Escalated Base Rent)')

ax.axvline(np.log(df['current_monthly_rent']).median(), color='steelblue', ls='--', lw=1.5)
ax.axvline(np.log(df['de_escalated_base_rent'].clip(lower=1)).median(), color='coral', ls='--', lw=1.5)

ax.set_title('Section 5.2.1: De-Escalated Base Rent vs Current Rent (log scale)')
ax.set_xlabel('log($/month)')
ax.set_ylabel('Frequency')
ax.legend()
plt.tight_layout()
plt.show()

esc_ratio = df['current_monthly_rent'] / df['de_escalated_base_rent'].clip(lower=1)
print(f'Escalation ratio (current / de-escalated):')
print(f'  Mean:   {esc_ratio.mean():.2f}x')
print(f'  Median: {esc_ratio.median():.2f}x')
print(f'  Range:  [{esc_ratio.min():.2f}x, {esc_ratio.max():.2f}x]')

### 2.6 Key Market Feature Distributions

These three features are among the strongest FMV drivers in the generating function:
- **census_population_density**: captures urban/rural placement
- **census_median_home_value**: the strongest single predictor of FMV
- **underlying_land_value_psf**: directly reflects local real estate economics

In [ ]:
market_features = [
    ('census_population_density', 'Pop. Density (persons/sq km)'),
    ('census_median_home_value',  'Median Home Value ($)'),
    ('underlying_land_value_psf', 'Land Value ($/sqft)'),
]

fig, axes = plt.subplots(2, 3, figsize=(16, 8))

for i, (col, label) in enumerate(market_features):
    # Raw histogram
    axes[0, i].hist(df[col], bins=80, color='teal', edgecolor='white', alpha=0.8)
    axes[0, i].set_title(f'{label}\n(raw)')
    axes[0, i].set_ylabel('Frequency')

    # Log-transformed histogram
    log_vals = np.log1p(df[col])
    axes[1, i].hist(log_vals, bins=80, color='darkorange', edgecolor='white', alpha=0.8)
    axes[1, i].set_title(f'{label}\n(log-transformed)')
    axes[1, i].set_xlabel(f'log(1 + value)')
    axes[1, i].set_ylabel('Frequency')

plt.suptitle('Section 5.2.1: Key Market Feature Distributions', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

for col, label in market_features:
    s = df[col]
    print(f'{col}: mean={s.mean():.1f}, median={s.median():.1f}, '
          f'skew={s.skew():.2f}, range=[{s.min():.1f}, {s.max():.1f}]')

---
## 3. Section 5.2.2 -- Bivariate Analysis

We explore pairwise relationships between rent and key predictors. Scatter plots
are colored by property type or escalation type to reveal conditional structure.

### 3.1 Current Rent vs Underlying Land Value (colored by Property Type)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    axes[0].scatter(df.loc[mask, 'underlying_land_value_psf'],
                    df.loc[mask, 'current_monthly_rent'],
                    s=8, alpha=0.35, label=pt, color=PROP_COLORS[pt])
axes[0].set_title('Current Rent vs Land Value (raw)')
axes[0].set_xlabel('Underlying Land Value ($/sqft)')
axes[0].set_ylabel('Current Monthly Rent ($/mo)')
axes[0].legend(fontsize=7, markerscale=2, ncol=2)

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    axes[1].scatter(np.log1p(df.loc[mask, 'underlying_land_value_psf']),
                    np.log(df.loc[mask, 'current_monthly_rent']),
                    s=8, alpha=0.35, label=pt, color=PROP_COLORS[pt])
axes[1].set_title('log(Rent) vs log(Land Value)')
axes[1].set_xlabel('log(1 + Land Value $/sqft)')
axes[1].set_ylabel('log(Rent $/mo)')

r_p, _ = pearsonr(np.log1p(df['underlying_land_value_psf']),
                   np.log(df['current_monthly_rent']))
r_s, _ = spearmanr(df['underlying_land_value_psf'], df['current_monthly_rent'])
axes[1].annotate(f'Pearson r = {r_p:.3f}\nSpearman rho = {r_s:.3f}',
                 xy=(0.05, 0.92), xycoords='axes fraction', fontsize=9,
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Section 5.2.2: Rent vs Underlying Land Value', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

### 3.2 Current Rent vs Census Population Density (colored by Property Type)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    axes[0].scatter(df.loc[mask, 'census_population_density'],
                    df.loc[mask, 'current_monthly_rent'],
                    s=8, alpha=0.35, label=pt, color=PROP_COLORS[pt])
axes[0].set_title('Current Rent vs Population Density (raw)')
axes[0].set_xlabel('Population Density (persons/sq km)')
axes[0].set_ylabel('Current Monthly Rent ($/mo)')
axes[0].legend(fontsize=7, markerscale=2, ncol=2)

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    axes[1].scatter(np.log1p(df.loc[mask, 'census_population_density']),
                    np.log(df.loc[mask, 'current_monthly_rent']),
                    s=8, alpha=0.35, label=pt, color=PROP_COLORS[pt])

r_p, _ = pearsonr(np.log1p(df['census_population_density']),
                   np.log(df['current_monthly_rent']))
r_s, _ = spearmanr(df['census_population_density'], df['current_monthly_rent'])
axes[1].annotate(f'Pearson r = {r_p:.3f}\nSpearman rho = {r_s:.3f}',
                 xy=(0.05, 0.92), xycoords='axes fraction', fontsize=9,
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1].set_title('log(Rent) vs log(Population Density)')
axes[1].set_xlabel('log(1 + Pop Density)')
axes[1].set_ylabel('log(Rent $/mo)')

plt.suptitle('Section 5.2.2: Rent vs Population Density', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

### 3.3 Current Rent vs Lease Age (Test H5: Older Leases More Dispersed?)

**Hypothesis H5**: Older leases should exhibit more dispersion in rent relative to FMV
because they have been subject to more cumulative escalation drift. We test this by
examining the spread (IQR) of rent within lease-age bins.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    axes[0].scatter(df.loc[mask, 'lease_age_years'],
                    df.loc[mask, 'current_monthly_rent'],
                    s=8, alpha=0.3, label=pt, color=PROP_COLORS[pt])
axes[0].set_title('Current Rent vs Lease Age')
axes[0].set_xlabel('Lease Age (years)')
axes[0].set_ylabel('Current Monthly Rent ($/mo)')
axes[0].legend(fontsize=7, markerscale=2, ncol=2)

# IQR by lease-age bins
df['_lease_age_bin'] = pd.cut(df['lease_age_years'], bins=10)
iqr_by_age = (df.groupby('_lease_age_bin', observed=True)['current_monthly_rent']
                .agg(lambda x: x.quantile(0.75) - x.quantile(0.25)))
bin_centers = [interval.mid for interval in iqr_by_age.index]
axes[1].bar(range(len(bin_centers)), iqr_by_age.values, color='mediumpurple', alpha=0.8)
axes[1].set_xticks(range(len(bin_centers)))
axes[1].set_xticklabels([f'{c:.0f}' for c in bin_centers], rotation=45)
axes[1].set_title('Rent IQR by Lease Age Bin (H5 Test)')
axes[1].set_xlabel('Lease Age Bin Center (years)')
axes[1].set_ylabel('IQR of Current Rent ($)')

abs_dev = np.abs(df['current_monthly_rent'] - df['current_monthly_rent'].median())
r_s_dev, p_s_dev = spearmanr(df['lease_age_years'], abs_dev)
axes[1].annotate(f'Spearman(age, |dev|) = {r_s_dev:.3f} (p={p_s_dev:.2e})',
                 xy=(0.05, 0.92), xycoords='axes fraction', fontsize=9,
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

df.drop(columns='_lease_age_bin', inplace=True)

plt.suptitle('Section 5.2.2: Lease Age vs Rent Dispersion (H5)', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

if r_s_dev > 0 and p_s_dev < 0.05:
    print(f'H5 SUPPORTED: Older leases have significantly more rent dispersion '
          f'(Spearman r={r_s_dev:.3f}, p={p_s_dev:.2e}).')
else:
    print(f'H5 NOT CONCLUSIVE: Spearman r={r_s_dev:.3f}, p={p_s_dev:.2e}.')

### 3.4 De-Escalated Base Rent vs Current Rent (colored by Escalation Type)

In [ ]:
esc_colors = {'fixed_3pct': '#e74c3c', 'fixed_2pct': '#3498db',
              'cpi_linked': '#2ecc71', 'flat': '#9b59b6'}

fig, ax = plt.subplots(figsize=(8, 6))

for et in ['fixed_3pct', 'fixed_2pct', 'cpi_linked', 'flat']:
    mask = df['escalation_type'] == et
    ax.scatter(df.loc[mask, 'de_escalated_base_rent'],
              df.loc[mask, 'current_monthly_rent'],
              s=8, alpha=0.35, label=et, color=esc_colors[et])

lims = [0, df[['current_monthly_rent', 'de_escalated_base_rent']].max().max() * 1.05]
ax.plot(lims, lims, 'k--', lw=1, label='y = x (no escalation)', alpha=0.5)

ax.set_title('Section 5.2.2: De-Escalated Base Rent vs Current Rent')
ax.set_xlabel('De-Escalated Base Rent ($/mo)')
ax.set_ylabel('Current Monthly Rent ($/mo)')
ax.legend(fontsize=8, markerscale=2)
plt.tight_layout()
plt.show()

for et in ['fixed_3pct', 'fixed_2pct', 'cpi_linked', 'flat']:
    mask = df['escalation_type'] == et
    ratio = (df.loc[mask, 'current_monthly_rent'] /
             df.loc[mask, 'de_escalated_base_rent'].clip(lower=1))
    print(f'{et:12s}: median escalation ratio = {ratio.median():.2f}x  (n={mask.sum()})')

### 3.5 Rent-to-Land-Value Ratio by Property Type

This ratio normalises rent by the underlying land value to see if certain property
types have systematically higher or lower rent efficiency.

In [ ]:
df['rent_to_land_value_ratio'] = (
    df['current_monthly_rent'] /
    (df['underlying_land_value_psf'] * df['lease_area_sqft'] / 12 + 1)
)

fig, ax = plt.subplots(figsize=(10, 5))
order = (df.groupby('property_type')['rent_to_land_value_ratio']
           .median().sort_values(ascending=False).index.tolist())
sns.boxplot(data=df, x='property_type', y='rent_to_land_value_ratio',
            order=order, palette=PROP_COLORS, ax=ax,
            flierprops=dict(marker='o', markersize=2, alpha=0.3))
ax.set_title('Section 5.2.2: Rent-to-Land-Value Ratio by Property Type')
ax.set_xlabel('Property Type')
ax.set_ylabel('Rent / (Land Value x Area / 12)')
ax.tick_params(axis='x', rotation=30)

ymax = df['rent_to_land_value_ratio'].quantile(0.99)
ax.set_ylim(0, ymax * 1.1)
plt.tight_layout()
plt.show()

ratio_summary = (df.groupby('property_type')['rent_to_land_value_ratio']
                   .agg(['median', 'mean', 'std'])
                   .round(3).sort_values('median', ascending=False))
print('\nRent-to-Land-Value Ratio by Property Type:')
ratio_summary

---
## 4. Section 5.2.3 -- Spatial Analysis

Cell tower lease values are inherently spatial -- rents correlate with local market
conditions. We visualise the geographic distribution of sites and examine how rent
varies across states and metro regions.

### 4.1 US Map -- Sites Colored by Rent and Property Type

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

sc1 = axes[0].scatter(df['lon'], df['lat'],
                      c=df['current_monthly_rent'], cmap='YlOrRd',
                      s=5, alpha=0.5,
                      vmin=df['current_monthly_rent'].quantile(0.05),
                      vmax=df['current_monthly_rent'].quantile(0.95))
plt.colorbar(sc1, ax=axes[0], label='Rent ($/mo)', shrink=0.8)
axes[0].set_title('Site Locations -- Colored by Rent')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_xlim(-130, -65)
axes[0].set_ylim(24, 50)
axes[0].set_aspect('equal')

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    axes[1].scatter(df.loc[mask, 'lon'], df.loc[mask, 'lat'],
                    s=5, alpha=0.45, label=pt, color=PROP_COLORS[pt])
axes[1].set_title('Site Locations -- Colored by Property Type')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_xlim(-130, -65)
axes[1].set_ylim(24, 50)
axes[1].set_aspect('equal')
axes[1].legend(fontsize=7, markerscale=3, loc='lower left', ncol=2)

plt.suptitle('Section 5.2.3: Geographic Distribution of Sites', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

### 4.2 Median Rent by State

In [ ]:
state_rent = (df.groupby('state')['current_monthly_rent']
                .agg(['median', 'count'])
                .sort_values('median', ascending=False))

fig, ax = plt.subplots(figsize=(14, 5))
colors = plt.cm.YlOrRd(np.linspace(0.3, 0.9, len(state_rent)))
ax.bar(range(len(state_rent)), state_rent['median'], color=colors, edgecolor='white')
ax.set_xticks(range(len(state_rent)))
ax.set_xticklabels(state_rent.index, rotation=45, ha='right', fontsize=9)
ax.set_title('Section 5.2.3: Median Monthly Rent by State')
ax.set_xlabel('State')
ax.set_ylabel('Median Rent ($/mo)')

for i, (_, row) in enumerate(state_rent.iterrows()):
    ax.text(i, row['median'] + 20, f'n={int(row["count"])}',
            ha='center', va='bottom', fontsize=7)

plt.tight_layout()
plt.show()

### 4.3 Median Rent by Metro Region

In [ ]:
metro_rent = (df.groupby('metro_region')['current_monthly_rent']
                .agg(['median', 'count'])
                .sort_values('median', ascending=True))

fig, ax = plt.subplots(figsize=(10, 10))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(metro_rent)))
ax.barh(range(len(metro_rent)), metro_rent['median'], color=colors, edgecolor='white')
ax.set_yticks(range(len(metro_rent)))
ax.set_yticklabels(metro_rent.index, fontsize=8)
ax.set_title('Section 5.2.3: Median Monthly Rent by Metro Region')
ax.set_xlabel('Median Rent ($/mo)')

for i, (_, row) in enumerate(metro_rent.iterrows()):
    ax.text(row['median'] + 10, i, f'n={int(row["count"])}',
            ha='left', va='center', fontsize=7)

plt.tight_layout()
plt.show()

---
## 5. Section 5.2.4 -- Cross-Property-Type Analysis

A critical architectural question: **should we build a unified model across all 7
property types, or separate type-specific models?** We answer this with:
1. Kruskal-Wallis test on rent across types
2. ANOVA on the rent-to-land-value ratio
3. Within-type vs across-type variance comparison

### 5.1 Kruskal-Wallis Test: Is Rent Significantly Different Across Property Types?

In [ ]:
groups = [df.loc[df['property_type'] == pt, 'current_monthly_rent'].values
          for pt in PROP_TYPES]

kw_stat, kw_pval = kruskal(*groups)
print('=' * 60)
print('Kruskal-Wallis Test: Current Monthly Rent ~ Property Type')
print('=' * 60)
print(f'H-statistic: {kw_stat:.2f}')
print(f'p-value:     {kw_pval:.2e}')
if kw_pval < 0.05:
    print('Conclusion:  REJECT H0 -- rent distributions differ significantly')
else:
    print('Conclusion:  FAIL TO REJECT H0')
print()

# Pairwise Mann-Whitney U tests
n_pairs = len(list(combinations(PROP_TYPES, 2)))
alpha_corrected = 0.05 / n_pairs
sig_pairs = 0
for pt1, pt2 in combinations(PROP_TYPES, 2):
    g1 = df.loc[df['property_type'] == pt1, 'current_monthly_rent'].values
    g2 = df.loc[df['property_type'] == pt2, 'current_monthly_rent'].values
    _, p_val = stats.mannwhitneyu(g1, g2, alternative='two-sided')
    if p_val < alpha_corrected:
        sig_pairs += 1

print(f'Pairwise Mann-Whitney U (Bonferroni-corrected):')
print(f'  {sig_pairs}/{n_pairs} pairs significantly different at alpha={alpha_corrected:.4f}')

### 5.2 ANOVA on Rent-to-Land-Value Ratio by Property Type

In [ ]:
ratio_groups = [df.loc[df['property_type'] == pt, 'rent_to_land_value_ratio'].dropna().values
                for pt in PROP_TYPES]

f_stat, f_pval = f_oneway(*ratio_groups)
print('=' * 60)
print('ANOVA: Rent-to-Land-Value Ratio ~ Property Type')
print('=' * 60)
print(f'F-statistic: {f_stat:.2f}')
print(f'p-value:     {f_pval:.2e}')
if f_pval < 0.05:
    print('Conclusion:  REJECT H0 -- ratio differs significantly across types')
else:
    print('Conclusion:  FAIL TO REJECT H0')

### 5.3 Within-Type vs Across-Type Variance Comparison

In [ ]:
portfolio_var = df['current_monthly_rent'].var()
portfolio_cv = df['current_monthly_rent'].std() / df['current_monthly_rent'].mean()

within_vars = []
within_cvs = []
for pt in PROP_TYPES:
    subset = df.loc[df['property_type'] == pt, 'current_monthly_rent']
    within_vars.append(subset.var())
    within_cvs.append(subset.std() / subset.mean())

avg_within_var = np.mean(within_vars)
avg_within_cv = np.mean(within_cvs)

group_means = df.groupby('property_type')['current_monthly_rent'].mean()
between_var = group_means.var()

# Eta-squared (effect size)
ss_between = sum(
    len(df[df['property_type'] == pt]) *
    (group_means[pt] - df['current_monthly_rent'].mean()) ** 2
    for pt in PROP_TYPES
)
ss_total = portfolio_var * (len(df) - 1)
eta_squared = ss_between / ss_total

print('=' * 60)
print('Variance Decomposition: Within vs Across Property Types')
print('=' * 60)
print(f'Portfolio CV:        {portfolio_cv:.4f}')
print(f'Avg Within-Type CV:  {avg_within_cv:.4f}')
print(f'Portfolio Variance:  {portfolio_var:,.0f}')
print(f'Avg Within-Type Var: {avg_within_var:,.0f}')
print(f'Between-Type Var:    {between_var:,.0f}')
print(f'Eta-squared:         {eta_squared:.4f} '
      f'({eta_squared*100:.1f}% of variance explained by property type)')
print()
print('Per-Type CV:')
for pt, cv in zip(PROP_TYPES, within_cvs):
    n = (df['property_type'] == pt).sum()
    print(f'  {pt:15s}: CV = {cv:.4f}  (n={n})')

In [ ]:
print('=' * 60)
print('UNIFIED MODELING ASSESSMENT')
print('=' * 60)
if eta_squared < 0.06:
    print(f'Eta-squared = {eta_squared:.4f} < 0.06: property type explains a SMALL '
          f'share of variance.')
    print('=> UNIFIED MODELING IS JUSTIFIED. Property type can be included as a feature')
    print('   rather than requiring separate models per type.')
elif eta_squared < 0.14:
    print(f'Eta-squared = {eta_squared:.4f}: property type explains a MODERATE share.')
    print('=> UNIFIED MODELING is viable but property_type interactions may help.')
else:
    print(f'Eta-squared = {eta_squared:.4f}: property type explains a LARGE share.')
    print('=> Consider SEPARATE MODELS per property type.')

print(f'\nMost variation is WITHIN types, not between them.')

---
## 6. Section 5.2.5 -- Statistical Tests

Formal statistical tests to validate modelling assumptions:
- Normality of log-rent (Shapiro-Wilk)
- Feature correlation structure (Pearson and Spearman heatmaps)
- Multicollinearity (VIF)
- Outlier detection (IQR method per property type)

### 6.1 Shapiro-Wilk Test on log(rent)

Shapiro-Wilk is limited to n < 5,000. We use a random subsample of 2,000 for a
reliable test.

In [ ]:
np.random.seed(42)
sample_size = 2000
sample_idx = np.random.choice(len(df), size=sample_size, replace=False)
log_rent_sample = np.log(df.iloc[sample_idx]['current_monthly_rent'])

sw_stat, sw_pval = shapiro(log_rent_sample)
print('=' * 60)
print(f'Shapiro-Wilk Test on log(rent), n={sample_size}')
print('=' * 60)
print(f'W-statistic: {sw_stat:.6f}')
print(f'p-value:     {sw_pval:.4e}')

if sw_pval < 0.05:
    print('Result: REJECT normality at alpha=0.05.')
    print('  Note: Shapiro-Wilk is very sensitive at n=2000. Small deviations from')
    print('  normality (e.g., slightly heavy tails) will trigger rejection.')
    print('  The Q-Q plot above may be a better practical guide.')
else:
    print('Result: FAIL TO REJECT normality at alpha=0.05.')
    print('  log(rent) is approximately normally distributed.')

### 6.2 Correlation Matrix (Top 20 Features)

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
exclude = ['true_fmv', 'true_deviation_pct', 'rent_to_land_value_ratio']
numeric_cols = [c for c in numeric_cols if c not in exclude]

corr_with_rent = (df[numeric_cols]
                  .corrwith(df['current_monthly_rent'], method='spearman').abs())
top_20 = corr_with_rent.sort_values(ascending=False).head(20).index.tolist()

print('Top 20 features by |Spearman rho| with current_monthly_rent:')
for i, feat in enumerate(top_20, 1):
    r = corr_with_rent[feat]
    print(f'  {i:2d}. {feat:40s} rho = {r:.3f}')

In [ ]:
corr_pearson = df[top_20].corr(method='pearson')
corr_spearman = df[top_20].corr(method='spearman')

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

mask_p = np.triu(np.ones_like(corr_pearson, dtype=bool))
sns.heatmap(corr_pearson, mask=mask_p, cmap='RdBu_r', center=0,
            annot=True, fmt='.2f', ax=axes[0], square=True,
            linewidths=0.5, annot_kws={'size': 6},
            cbar_kws={'shrink': 0.8, 'label': 'Pearson r'})
axes[0].set_title('Pearson Correlation Matrix (Top 20)', fontsize=11)
axes[0].tick_params(axis='both', labelsize=7)

mask_s = np.triu(np.ones_like(corr_spearman, dtype=bool))
sns.heatmap(corr_spearman, mask=mask_s, cmap='RdBu_r', center=0,
            annot=True, fmt='.2f', ax=axes[1], square=True,
            linewidths=0.5, annot_kws={'size': 6},
            cbar_kws={'shrink': 0.8, 'label': 'Spearman rho'})
axes[1].set_title('Spearman Correlation Matrix (Top 20)', fontsize=11)
axes[1].tick_params(axis='both', labelsize=7)

plt.suptitle('Section 5.2.5: Correlation Matrices', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

### 6.3 VIF Analysis (Top 15 Continuous Features)

Variance Inflation Factor measures multicollinearity. VIF > 5 flags features that
are well-explained by other features, which can inflate standard errors in linear
models (though tree-based models are largely immune).

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

continuous_candidates = [c for c in top_20 if df[c].nunique() > 10]
vif_features = continuous_candidates[:15]

X_vif = df[vif_features].fillna(0).values.astype(float)
X_vif = (X_vif - X_vif.mean(axis=0)) / (X_vif.std(axis=0) + 1e-10)

vif_data = pd.DataFrame({
    'Feature': vif_features,
    'VIF': [variance_inflation_factor(X_vif, i) for i in range(X_vif.shape[1])]
}).sort_values('VIF', ascending=False)

print('=' * 60)
print('VIF Analysis (Top 15 Continuous Features)')
print('=' * 60)
n_high = 0
for _, row in vif_data.iterrows():
    flag = ' *** HIGH' if row['VIF'] > 5 else ''
    if row['VIF'] > 5:
        n_high += 1
    print(f'  {row["Feature"]:40s} VIF = {row["VIF"]:8.2f}{flag}')

print(f'\n{n_high} feature(s) with VIF > 5 flagged for potential multicollinearity.')
if n_high > 0:
    print('Note: Tree-based models (XGBoost, RF) are robust to multicollinearity.')
    print('High VIF features may still be informative but should not be used in')
    print('linear models without regularisation.')

### 6.4 Outlier Detection (IQR Method per Property Type)

We identify rent outliers within each property type using the IQR fence method:
a site is flagged if its rent falls below Q1 - 1.5 x IQR or above Q3 + 1.5 x IQR.

In [ ]:
outlier_flags = pd.Series(False, index=df.index)
outlier_records = []

for pt in PROP_TYPES:
    mask = df['property_type'] == pt
    rents = df.loc[mask, 'current_monthly_rent']
    Q1 = rents.quantile(0.25)
    Q3 = rents.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    is_outlier = (rents < lower) | (rents > upper)
    outlier_flags.loc[is_outlier[is_outlier].index] = True

    outlier_records.append({
        'Property Type': pt, 'N': mask.sum(),
        'Q1': round(Q1), 'Q3': round(Q3), 'IQR': round(IQR),
        'Lower Fence': round(lower), 'Upper Fence': round(upper),
        'Outliers': is_outlier.sum(),
        'Outlier %': round(is_outlier.sum() / mask.sum() * 100, 1),
    })

outlier_df = pd.DataFrame(outlier_records).set_index('Property Type')

print('=' * 60)
print('IQR-Based Outlier Detection by Property Type')
print('=' * 60)
print(f'Total outliers: {outlier_flags.sum()} / {len(df)} '
      f'({outlier_flags.mean()*100:.1f}%)')
print()
outlier_df

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

non_outlier = df[~outlier_flags]
outlier_sites = df[outlier_flags]

ax.scatter(non_outlier['lon'], non_outlier['lat'],
           s=3, alpha=0.3, c='grey', label=f'Normal (n={len(non_outlier)})')
ax.scatter(outlier_sites['lon'], outlier_sites['lat'],
           s=15, alpha=0.7, c='red', marker='x',
           label=f'IQR Outliers (n={len(outlier_sites)})')
ax.set_title('Section 5.2.5: Geographic Distribution of Rent Outliers')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_xlim(-130, -65)
ax.set_ylim(24, 50)
ax.set_aspect('equal')
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

---
## 7. Section 5.2.6 -- Temporal Analysis

Leases accumulate escalation over time. The key question: **does the contractual
escalation match inflation?** We define *escalation drift* as the ratio of the
cumulative escalation factor to the expected CPI factor.

### 7.1 Lease Age vs Cumulative Escalation Factor (by Escalation Type)

In [ ]:
esc_colors = {'fixed_3pct': '#e74c3c', 'fixed_2pct': '#3498db',
              'cpi_linked': '#2ecc71', 'flat': '#9b59b6'}

fig, ax = plt.subplots(figsize=(10, 6))

for et, color in esc_colors.items():
    mask = df['escalation_type'] == et
    ax.scatter(df.loc[mask, 'lease_age_years'],
              df.loc[mask, 'cumulative_escalation_factor'],
              s=8, alpha=0.4, label=et, color=color)

# Theoretical curves
age_grid = np.linspace(0, 25, 100)
for rate, label, ls in [(0.03, '3% fixed', '--'), (0.02, '2% fixed', ':'),
                         (0.025, 'CPI ~2.5%', '-.'), (0.0, 'Flat', '-')]:
    ax.plot(age_grid, (1 + rate) ** age_grid, 'k', ls=ls, lw=0.8, alpha=0.5)

ax.set_title('Section 5.2.6: Lease Age vs Cumulative Escalation Factor')
ax.set_xlabel('Lease Age (years)')
ax.set_ylabel('Cumulative Escalation Factor')
ax.legend(fontsize=8, markerscale=2)
plt.tight_layout()
plt.show()

### 7.2 Escalation Drift Computation

We compute `escalation_drift = cumulative_escalation_factor / expected_cpi_factor`,
where expected CPI factor approximates 25-year average CPI at ~2.5%/year.

- **drift > 1.0**: lease has escalated faster than CPI (overheating)
- **drift < 1.0**: lease has escalated slower than CPI (falling behind)
- **drift = 1.0**: escalation perfectly matches CPI

In [ ]:
# Compute expected CPI factor (approximate 2.5% per year)
expected_cpi_factor = (1.025) ** df['lease_age_years']

# Compute escalation drift
escalation_drift = df['cumulative_escalation_factor'] / expected_cpi_factor

# Dedicated escalation drift histogram
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Overall drift distribution
axes[0].hist(escalation_drift, bins=80, color='steelblue', edgecolor='white', alpha=0.8)
axes[0].axvline(1.0, color='red', ls='--', lw=1.5, label='Drift = 1.0 (CPI-neutral)')
axes[0].axvline(escalation_drift.median(), color='orange', ls='--', lw=1.5,
                label=f'Median = {escalation_drift.median():.3f}')
axes[0].set_title('Escalation Drift Distribution (All Sites)')
axes[0].set_xlabel('Drift = Cumulative Factor / Expected CPI Factor')
axes[0].set_ylabel('Frequency')
axes[0].legend(fontsize=8)

# Drift by escalation type (faceted)
for et, color in esc_colors.items():
    mask = df['escalation_type'] == et
    axes[1].hist(escalation_drift[mask], bins=40, alpha=0.5,
                 label=et, color=color, edgecolor='none')
axes[1].axvline(1.0, color='red', ls='--', lw=1.5)
axes[1].set_title('Escalation Drift by Escalation Type')
axes[1].set_xlabel('Drift = Cumulative Factor / Expected CPI Factor')
axes[1].set_ylabel('Frequency')
axes[1].legend(fontsize=8)

plt.suptitle('Section 5.2.6: Escalation Drift Analysis', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print('Escalation Drift Statistics:')
print(f'  Mean:   {escalation_drift.mean():.3f}')
print(f'  Median: {escalation_drift.median():.3f}')
print(f'  Std:    {escalation_drift.std():.3f}')
print(f'  % above CPI (drift > 1): {(escalation_drift > 1).mean()*100:.1f}%')
print(f'  % below CPI (drift < 1): {(escalation_drift < 1).mean()*100:.1f}%')

In [ ]:
print('Escalation Drift by Type:')
for et in ['fixed_3pct', 'fixed_2pct', 'cpi_linked', 'flat']:
    mask = df['escalation_type'] == et
    drift = escalation_drift[mask]
    print(f'  {et:12s}: median drift = {drift.median():.3f}, '
          f'mean = {drift.mean():.3f}, '
          f'% above CPI = {(drift > 1).mean()*100:.1f}%  (n={mask.sum()})')

---
## 8. Summary and Key Findings

In [ ]:
print('=' * 70)
print('COMPREHENSIVE EDA -- KEY FINDINGS')
print('=' * 70)

print(f'''
1. RENT DISTRIBUTION
   - Current monthly rent is right-skewed (skew={df['current_monthly_rent'].skew():.2f}).
   - Log-transform substantially normalises the distribution.
   - Median rent: ${df['current_monthly_rent'].median():,.0f}/mo,
     range: ${df['current_monthly_rent'].min():,.0f} - ${df['current_monthly_rent'].max():,.0f}/mo.

2. PROPERTY TYPE EFFECTS
   - Kruskal-Wallis test: rent distributions differ significantly across types.
   - Eta-squared = {eta_squared:.4f}: property type explains ~{eta_squared*100:.1f}% of variance.
   - Most variation is WITHIN property types, not between them.
   => UNIFIED MODELING IS JUSTIFIED with property_type as a feature.

3. KEY PREDICTORS
   - census_median_home_value: strongest single predictor of rent.
   - census_population_density and underlying_land_value_psf: strong log-linear
     relationships with rent.
   - Land-value ratio varies moderately by property type.

4. SPATIAL PATTERNS
   - Clear metro clustering with expected rent gradients (coastal > inland > rural).
   - {df['state'].nunique()} states represented with adequate geographic diversity.

5. LEASE AGE AND ESCALATION
   - Lease ages: {df['lease_age_years'].min():.0f} - {df['lease_age_years'].max():.0f} years.
   - Fixed 3% escalation produces the largest drift above CPI.
   - Flat leases drift below CPI over time.
   - H5 (older leases more dispersed): {'SUPPORTED' if (r_s_dev > 0 and p_s_dev < 0.05) else 'NOT CONCLUSIVE'}.

6. MULTICOLLINEARITY
   - {n_high} features with VIF > 5 detected.
   - Tree-based models (XGBoost, QRF) are robust to this.

7. OUTLIERS
   - IQR method flags {outlier_flags.sum()} sites ({outlier_flags.mean()*100:.1f}%) as rent outliers.
   - Outliers are geographically dispersed (not concentrated in one region).

=> PROCEED TO CLUSTERING AND PSEUDO-LABELING (Notebook 02).
''')

In [ ]:
# Clean up temporary columns
if 'rent_to_land_value_ratio' in df.columns:
    df.drop(columns=['rent_to_land_value_ratio'], inplace=True)

print('EDA complete. Proceed to Notebook 02 for clustering and pseudo-labeling.')